# Using word2vec to see what CBC 'knows' about Canada

### or 

# What happened when I forced my computer to read 135,000 news stories

---

A talk at PyCon Canada 2019 by Roberto Rocha

Investigative data journalist at CBC


Twitter/Github: robroc

---

First things first. I'm not a real data scientist. I'm data journalist who uses some data science tools. I don't do predictive models. My job is not to predict, but to analyze and report. I'm by no means a machine learning practioner. So I have a very shallow understanding of this stuff.

What follows are experiments in using a text-based deep learning tool to get insights out of mountains of text. Specifically, as many past articles published on cbc.ca/news that I could get my hands on. And this was about 135,000 stories from 2014 to mid 2019.

This notebook is almost a complete copy of a [word2vec tutorial on Kaggle](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial), with a few modifications. That gentleman deserves most of the credit.

Also, I can't share the source data or the trained model. Not yet, anyway. So please don't ask. Sorry.

# What is word2vec? 

## **Short answer:** 
### A way to turn words into numbers based on their context and put them in a high-dimensional vector space



Here's an illustration of a word space reduced to three dimensions by PCA. [Taken from here](https://projector.tensorflow.org/).

![vector](images\word2vec.gif "3dvector")

## **Shorter answer:**


<img src="images\giphy.gif" alt="magic" style="width: 400px;"/>

Yes, this is the most magical thing I've seen a computer do. If you disagree, fight me.

## What is it used for?


* Recommendation engines
* Document similarity
* Bias detection


A [word2vec model trained on Google News articles](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf) found some signs of bias. I was wondering if CBC would fare better.

![bias](images/study.PNG)

### On to the code. First, load the libraries

In [1]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
import unicodedata
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

### Load in the CBC articles from a pandas dataframe. There's one article per row.

In [91]:
archive = pd.read_csv('cbc_stories_with_date_2014-19.csv', parse_dates = ['pubdate'])
stories = archive.body.tolist()
stories = [unicodedata.normalize("NFKD", s).encode('ASCII', 'ignore').decode().strip() for s in stories if pd.notna(s)]

C:\Users\Beto\AppData\Local\Continuum\miniconda3\envs\word2vec\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Here's what one story looks like:

In [13]:
stories[25]

'The New Brunswick government will have a razor-thin $4.5 million budget surplus this year, Progressive Conservative Finance Minister Ernie Steeves has announced.  The dramatic improvement over the $188.7-million deficit projected in the Liberal government\'s last budget is attributed mostly to higher than expected revenues, including personal and corporate income taxes and federal transfer payments. The new projection is from the government\'s third-quarter financial results for 2018-19. Premier Blaine Higgs let the cat out of the bag two weeks ago when he said the province would have a balanced budget this year. But Steeves warned that the surplus is on a "thin edge" and the province\'s debt continues to grow. That means his first provincial budget next month will look for new ways to reduce spending, he said. "It\'s going to be a difficult budget. We need to explore new ways to do things in a more efficient manner." Steeves said he\'s looking at all programs "from beginning to end" 

In [19]:
# Number of stories:
len(stories)

134997

In [20]:
# Number of words:
sum([len(txt) for txt in stories])

309522810

In [215]:
# Number of stories per publicaton year
archive.pubdate.dt.year.value_counts(sort = False)

2005        2
2007        3
2008        1
2009        1
2011        1
2012        3
2013        9
2014    30002
2015    30011
2016    30069
2017    30202
2018      428
2019    14811
Name: pubdate, dtype: int64

## The pre-processing

The cleanup is divided in two parts:
    
1. Split stories into sentences and remove non-alpha characters
2. Remove stop-words (and, I, or, with) and optionally lemmatize tokens

#### Part 1

Load the SpaCy English model and sentencizer, split stories into sentences, and do a quick cleaning: remove non-alphanumeric character and trailing spaces.

The SpaCy sentencizer smartly detects sentences beyond just splitting on periods. This avoids creating sentences out of "Mr." or acronyms.

In [32]:
nlp = spacy.load('en_core_web_sm', disable=['parser']) 
nlp.add_pipe(nlp.create_pipe('sentencizer'))

sents = []

t = time()

for story in stories: 
    s = [re.sub("[^A-zÀ-ú']+", ' ', sent.string).strip() for sent in nlp(story).sents]
    sents.extend(s)

print('Time to sentensize: {} mins'.format(round((time() - t) / 60, 2)))

Time to sentensize: 107.93 mins


This is was the cleaned up senteces look like:

In [36]:
sents[10:20]

['At the end of the week one photo submission wins a CBC London prize pack',
 "Currently in London it's and cloudy",
 "Today's forecast is mainly cloudy with a percent chance of flurries",
 'Winds are west between and km hr',
 'The temperature is steady at with a wind chill near Gas is selling in London between and per litre',
 'Source gasbuddy com The Canadian dollar closed yesterday at cents U S The market opens today at a m',
 "A report from the Toronto law firm tasked with investigating harassment at the City of London has found that nearly city employees say they've been harassed discriminated against bullied intimidated or experienced reprisal in the workplace",
 'Staff at Rubin Thomlinson LLP sent surveys to city of London employees and received back',
 "Of those responses current and former employees confirmed that they'd experienced this type of behaviour",
 'Intimidation was the most common behaviour experienced by city staff followed by bullying and harassment the report fou

#### Part 2

Now we need to clean individual tokens in the sentences: Remove stop-words and lemmatize words that are not a named entity.

Lemmatization is controversial. Some say it removes nuances. But for me, I wanted to reduce noise. I didn't care to keep plurals or the conjugated forms of verbs. However, I wanted to maintain some resolution. I didn't want Apple the company to be reduced to apple the fruit. Luckily, SpaCy has a handy named entity recognition function that identifies organizations, proper names and places so they don't get lost.

In [62]:
def cleaning(doc):
    """"Lemmatizes (optional) and removes stopwords
    `doc` needs to be a spacy Doc object
    """
    
    # Lemmatize words only if they're not a recognized entity type and skip stopwords
    txt = [token.lemma_ if token.ent_type == 0 else token.text for token in doc if not token.is_stop]
    
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(sents, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 107.92 mins


And here are the lemmatized and refined sentences.

In [129]:
txt[10:20]

['at end week photo submission win CBC London prize pack',
 'currently London be cloudy',
 "Today 's forecast mainly cloudy percent chance flurry",
 'wind west km hr',
 'the temperature steady wind chill near Gas sell London litre',
 'source gasbuddy com the Canadian dollar close yesterday cent U S the market open today m',
 'a report Toronto law firm task investigate harassment City London find nearly city employee have harass discriminate bully intimidate experience reprisal workplace',
 'staff Rubin Thomlinson LLP send survey city London employee receive',
 'of response current employee confirm would experience type behaviour',
 'intimidation common behaviour experience city staff follow bully harassment report find']

## Identify bigrams

Now we run the sentences through gensim's phraser. This is useful to catch bigrams (two-word combos that go together frequently) that mean different things.

For example, we'll want to differentiate Justin Trudeau from Trudeau airport, or Doug Ford from Rob Ford.

Finally, we want to split sentences into their component words. That's what wod2vec needs as input: a list of lists of words in a sentence.

In [ ]:
from gensim.models.phrases import Phrases, Phraser
# More info: https://radimrehurek.com/gensim/models/phrases.html

token_lists = [line.split() for line in txt if line]
phrases = Phrases(token_lists, min_count=20, progress_per=10000)

bigram = Phraser(phrases)
sentences = bigram[token_lists]

And here's the final result: a list of words in a sentence.

In [130]:
sentences.corpus[10]

['at',
 'end',
 'week',
 'photo',
 'submission',
 'win',
 'CBC',
 'London',
 'prize',
 'pack']

## The training

#### Initialize word2vec, keeping most default parameters.

In [71]:
import multiprocessing
from gensim.models import Word2Vec

cores = multiprocessing.cpu_count()

w2v_model = Word2Vec(min_count = 10,
                     window = 5,
                     size = 200,
                     workers=cores-1
                    )

Illustraton of the `window` parameter: how many neighbouring words to consider for the context. In this case, the window size is 2.

Source: [Chis McCormick](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

![window](images/training_data.png "window")

#### Build the vocabulary

In [ ]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#### Train the model (finally!)

In [ ]:
t = time()

w2v_model.train(sentences, total_examples = w2v_model.corpus_count, epochs=30, report_delay=1)

w2v_model.init_sims(replace=True)

Save the models for later. In this case, it's a KeyedVetor format that loads faster and uses less memory if I'm done training it.

In [ ]:
w2v_model.wv.save('word2vec5.wv')

## Evaluate the model

Now we can test how good the model is. A word like Toronto should be most similar to other large Canadian cities.

In [216]:
w2v_model.wv.most_similar('Toronto')

[('Montreal', 0.7378078103065491),
 ('Hamilton', 0.725394070148468),
 ('Vancouver', 0.7078638076782227),
 ('London', 0.6668410301208496),
 ('Ottawa', 0.663960337638855),
 ('Calgary', 0.640132486820221),
 ('Edmonton', 0.6318969130516052),
 ('Winnipeg', 0.6047239899635315),
 ('windsor', 0.5858883261680603),
 ('Guelph', 0.55284583568573)]

And it should also know that American cities are more similar to other American cities.

In [218]:
w2v_model.wv.most_similar('Chicago')

[('Philadelphia', 0.7164496779441833),
 ('Boston', 0.6958774328231812),
 ('Los_Angeles', 0.685975193977356),
 ('Pittsburgh', 0.6837000846862793),
 ('St_Louis', 0.6808935403823853),
 ('Nashville', 0.6802417039871216),
 ('Atlanta', 0.6772310733795166),
 ('Dallas', 0.6682097911834717),
 ('Seattle', 0.6489051580429077),
 ('Tampa', 0.617918848991394)]

Test other words that you know are similar. They should have high cosine similarity scores

In [175]:
word_pairs = [
    ('surgeon', 'doctor'),
    ('Canada', 'country'),
    ('blue', 'red'),
    ('movie', 'film'),
    ('dog', 'cat'),
    ('Liberal', 'Conservative')
]

for pair in word_pairs:
    print(pair, w2v_model.wv.similarity(*pair) )

('surgeon', 'doctor') 0.6474745
('Canada', 'country') 0.7405826
('blue', 'red') 0.6960444
('movie', 'film') 0.8581524
('dog', 'cat') 0.7754835
('Liberal', 'Conservative') 0.7897476


Let's see if the named entity recognizer did its job:

In [219]:
w2v_model.wv.most_similar('apple')

[('strawberry', 0.6267320513725281),
 ('sweet_potato', 0.6007342338562012),
 ('potato', 0.5943686366081238),
 ('broccoli', 0.5879802703857422),
 ('fruit', 0.5761351585388184),
 ('citrus', 0.5758861303329468),
 ('carrot', 0.5754052400588989),
 ('caramel', 0.5738547444343567),
 ('pecan', 0.5734065771102905),
 ('pc_organics', 0.5699262619018555)]

In [200]:
w2v_model.wv.most_similar('Apple')

[('Apple_Inc', 0.7471168041229248),
 ('Microsoft', 0.7063766717910767),
 ('Samsung', 0.6787918210029602),
 ('BlackBerry', 0.6776144504547119),
 ('iPhone', 0.6532427072525024),
 ('Apple_Google', 0.6327992677688599),
 ('Google', 0.6271075010299683),
 ('Android', 0.6246681809425354),
 ('iPhones', 0.5998338460922241),
 ('Amazon', 0.5953640341758728)]

### Analogies!

This is the coolest feature of word2vec. Since words were converted to numbers, we can do math with them. We can, for example, subtract one word from a pair of words and get the semantic equivalent.

In other words we can as questions like: what is to woman as king is to man?

In [201]:
w2v_model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.4101138710975647),
 ('prince', 0.40189918875694275),
 ('William_Kate', 0.39393043518066406),
 ('princess', 0.37827563285827637),
 ('emperor', 0.36916765570640564),
 ('king_Salman', 0.36641332507133484),
 ('connolly', 0.35293644666671753),
 ('Kertesz', 0.35092097520828247),
 ('luther', 0.3470686078071594),
 ('pope', 0.3422856330871582)]

Since this is a corpus of Canadian news, let's see what CBC knows about Canadian cities!

In [222]:
cities = ['Toronto', 'Montreal', 'Vancouver', 'Ottawa', 
          'Halifax', 'Calgary', 'Winnipeg']

# Function to get the equivalent of a given word and city, for the other cities above.
def city_analogies(word, city2):
    """Find the top five words that are to a city as word is to city2"""
    
    df = pd.DataFrame(columns = cities)
    for city in cities:
        if city == city2:
            continue
        sims = [res[0] for res in w2v_model.wv.most_similar(positive=[city, word], negative=[city2], topn=5)]
        df[city] = sims
    return df

# What are the other cities' equivalent of poutine?
city_analogies('poutine', 'Montreal')

,Toronto,Montreal,Vancouver,Ottawa,Halifax,Calgary,Winnipeg
0,grill_cheese,NaN,sushi,bacon,pita,apple_pie,burrito
1,condiment,NaN,condiment,burrito,meatless,latte,hearty
2,dessert,NaN,grill_cheese,condiment,pizza,burger,burger
3,burger,NaN,gourmet,grill_cheese,porridge,bacon,mac_cheese
4,bacon,NaN,dessert,nachos,donut,perogie,condiment


In [223]:
# What about Drake and Toronto?

city_analogies('Drake', 'Toronto')

,Toronto,Montreal,Vancouver,Ottawa,Halifax,Calgary,Winnipeg
0,NaN,Leonard_Cohen,rapper,Celine_Dion,N_S,Taylor_Swift,Man
1,NaN,celine,Swift,Classified,Kinley,Thriller,Taylor_Swift
2,NaN,drake,Bob_Dylan,Weeknd,Classified,Jay_Z,drake
3,NaN,Celine_Dion,Taylor_Swift,Alanis_Morissette,lower_sackville,drake,Janet_Jackson
4,NaN,rapper,Weeknd,drake,Kentville,Weeknd,hip_hop


In [229]:
# The Calgary Stampede?

city_analogies('Calgary_Stampede', 'Calgary')

,Toronto,Montreal,Vancouver,Ottawa,Halifax,Calgary,Winnipeg
0,extravaganza,Old_Port,extravaganza,Parliament_Hill,tall_ship,NaN,the_Forks
1,Harbourfront_Centre,fete,Harbourfront_Centre,Canada_Day,Yacht_Club,NaN,Gimli
2,th_edition,extravaganza,Wine_Festival,th_edition,Citadel_Hill,NaN,Voyageur
3,junos,th_edition,festival,Rideau_Canal,Eastlink_Centre,NaN,Handsome
4,telecast,festival,th_edition,Rideau_Hall,Lunenburg,NaN,Stampede


## Check for biases

Any gender biases hidden in CBC stories?

In [230]:
w2v_model.wv.most_similar(positive=['woman','computer_programmer'], negative=['man'], topn=6)

[('software_engineer', 0.47878366708755493),
 ('programmer', 0.45141080021858215),
 ('coder', 0.4328458905220032),
 ('illustrator', 0.4310019314289093),
 ('designer', 0.4219575524330139),
 ('software_developer', 0.4206082224845886)]

In [232]:
w2v_model.wv.most_similar(positive=['woman','ceo'], negative=['man'], topn=6)

[('president_ceo', 0.6393610239028931),
 ('chief_executive', 0.6163231134414673),
 ('vice_president', 0.558792769908905),
 ('interim_ceo', 0.5408685207366943),
 ('executive_director', 0.5326038002967834),
 ('president', 0.5301772356033325)]

Not bad. What about the reverse?

In [231]:
w2v_model.wv.most_similar(positive=['man','nurse'], negative=['woman'], topn=6)

[('paramedic', 0.5126073956489563),
 ('correction_officer', 0.49881601333618164),
 ('doctor', 0.4887685179710388),
 ('doctor_nurse', 0.4882134795188904),
 ('correctional_officer', 0.4696846604347229),
 ('jail_guard', 0.46581488847732544)]

That's interesting!

## Compare Harper and Trudeau governments

Are there differences in word associations in different governments? I trained the same model on 60,000 stories each, one during the Harper years and another during a comparable span of the Trudeau admnistration.

In [233]:
# For transparency, here is how the data was divided by date.

harper = archive[archive.pubdate <= '2015-10-20'].body.dropna().tolist()
trudeau = archive[(archive.pubdate >= '2015-10-21') & (archive.pubdate <= '2017-10-01')].body.dropna().tolist()

print(len(harper))
print(len(trudeau))

59904
60154


Below are the trained models:

In [172]:
trudeau_model.wv.most_similar('pipeline')

[('oil_pipeline', 0.7849512696266174),
 ('pipeline_project', 0.6998418569564819),
 ('Keystone_XL', 0.6799078583717346),
 ('Trans_Mountain', 0.6449182033538818),
 ('Energy_East', 0.6427786946296692),
 ('pipeline_expansion', 0.6376286745071411),
 ('TransCanada', 0.6276552677154541),
 ('Kinder_Morgan', 0.5928761959075928),
 ('kilometre_pipeline', 0.5828524827957153),
 ('Dakota_Access', 0.5747994184494019)]

In [170]:
harper_model.wv.most_similar('pipeline')

[('oil_pipeline', 0.7015752792358398),
 ('TransCanada', 0.6530065536499023),
 ('pipeline_project', 0.6472420692443848),
 ('Pipeline', 0.6225021481513977),
 ('Keystone_XL', 0.6192481517791748),
 ('pipeline_carry', 0.6157941818237305),
 ('gas_pipeline', 0.5762631893157959),
 ('oilsand', 0.5699912309646606),
 ('oil_sand', 0.569394588470459),
 ('enbridge', 0.567440390586853)]

In [235]:
trudeau_model.wv.most_similar('war')

[('civil_war', 0.5777413845062256),
 ('War', 0.5279940366744995),
 ('jihad', 0.5268000364303589),
 ('enemy', 0.507621705532074),
 ('Kurds', 0.49582090973854065),
 ('battlefield', 0.4905870854854584),
 ('Rwanda', 0.4848124086856842),
 ('Syrian_civil', 0.4839516282081604),
 ('Yugoslavia', 0.4831847548484802),
 ('Soviet_Union', 0.4819676876068115)]

In [234]:
harper_model.wv.most_similar('war')

[('conflict', 0.564337432384491),
 ('war_Syria', 0.5504180192947388),
 ('jihad', 0.5458228588104248),
 ('War', 0.5448305010795593),
 ('civil_war', 0.5056760311126709),
 ('military_mission', 0.4953949451446533),
 ('First_World', 0.4877474308013916),
 ('Afghanistan', 0.4829769730567932),
 ('battlefield', 0.4757120609283447),
 ('insurgency', 0.47483381628990173)]

### Visualizing the model

You can export your model and visualize it on https://projector.tensorflow.org/

In [ ]:
from gensim.scripts import word2vec2tensor

with open('cbc_tensor.tsv', 'w+') as file_vector:
    with open('cbc_tensor_metadata.tsv', 'w+') as file_metadata:
        for word in w2v_model.wv.index2word:
            file_metadata.write(gensim.utils.to_unicode(word) + gensim.utils.to_unicode('\n'))
            vector_row = '\t'.join(str(x) for x in w2train_modelsel[word])
            file_vector.write(vector_row + '\n')